In [ ]:
import os
from pathlib import Path
import shutil

def find_annotated_images(image_dir, annotation_dir, image_extension='.tiff', annotation_extension='.txt'):
    """
    Find images that have corresponding YOLO annotation files.

    Args:
        image_dir (str or Path): Path to the directory containing images.
        annotation_dir (str or Path): Path to the directory containing annotation files.
        image_extension (str): Extension of image files.
        annotation_extension (str): Extension of annotation files.

    Returns:
        annotated_images (list of Path): List of image paths that have annotations.
        non_annotated_images (list of Path): List of image paths without annotations.
    """
    image_dir = Path(image_dir)
    annotation_dir = Path(annotation_dir)

    # 获取所有图片文件
    all_images = list(image_dir.glob(f'*{image_extension}'))
    print(f"找到的总图片数量: {len(all_images)}")

    # 获取所有标注文件，排除 classes.txt
    annotation_files = [f for f in annotation_dir.glob(f'*{annotation_extension}') if f.name != 'classes.txt']
    print(f"找到的标注文件数量: {len(annotation_files)}")

    # 提取有标注的图片的基本名称
    annotated_basenames = set(f.stem for f in annotation_files)
    print(f"有标注的图片数量: {len(annotated_basenames)}")

    # 分离有标注和无标注的图片
    annotated_images = [img for img in all_images if img.stem in annotated_basenames]
    non_annotated_images = [img for img in all_images if img.stem not in annotated_basenames]

    print(f"有标注的图片: {len(annotated_images)}")
    print(f"无标注的图片: {len(non_annotated_images)}")

    return annotated_images, non_annotated_images

def organize_data(image_dir, annotation_dir, output_dir, annotated_images, non_annotated_images):
    """
    将图片和标注文件组织到不同的目录中。

    Args:
        image_dir (Path): 图片目录路径。
        annotation_dir (Path): 标注文件目录路径。
        output_dir (Path): 输出的组织后数据的目录路径。
        annotated_images (list of Path): 有标注的图片路径列表。
        non_annotated_images (list of Path): 无标注的图片路径列表。
    """
    output_dir = Path(output_dir)
    annotated_images_dir = output_dir / 'annotated_images'
    annotated_labels_dir = output_dir / 'annotated_labels'
    non_annotated_images_dir = output_dir / 'non_annotated_images'

    # 创建目标目录
    annotated_images_dir.mkdir(parents=True, exist_ok=True)
    annotated_labels_dir.mkdir(parents=True, exist_ok=True)
    non_annotated_images_dir.mkdir(parents=True, exist_ok=True)

    # 复制有标注的图片和对应的标注文件
    for img_path in annotated_images:
        shutil.copy(img_path, annotated_images_dir / img_path.name)
        label_path = annotation_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            shutil.copy(label_path, annotated_labels_dir / label_path.name)

    # 复制无标注的图片
    for img_path in non_annotated_images:
        shutil.copy(img_path, non_annotated_images_dir / img_path.name)

    print(f"有标注的图片已复制到: {annotated_images_dir}")
    print(f"有标注的标注文件已复制到: {annotated_labels_dir}")
    print(f"无标注的图片已复制到: {non_annotated_images_dir}")

def main():
    # 定义目录路径
    image_dir = "./TS-20241123222043114.avi_frames"
    annotation_dir = "./TS-20241123222043114.avi_frames/annotations_yolo"
    output_dir = "./organized_data"

    # 查找有标注和无标注的图片
    annotated_images, non_annotated_images = find_annotated_images(image_dir, annotation_dir)

    # 组织数据
    organize_data(image_dir, annotation_dir, output_dir, annotated_images, non_annotated_images)

main()
